Misspellings of curse words?  

When building vocab, consider punctuation. Tokenize it first? Right now it just splits on whitespace. Moses probably has something for this

In [40]:
from time import time


import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
np.random.seed(1234) # help reproducibility
olidData = 'data/OLIDv1.0/olid-training-v1.0.tsv'
validation = 'data/OLIDv1.0/testset-levela.tsv'





In [46]:
olid = open(olidData)
df = pd.read_csv(olid, sep='\t')
olid.close()
df = df.drop(df.columns[[0,3,4]], 1)


In [44]:
valid = open(validation)
validDf = df = pd.read_csv(valid, sep='\t')
validDf = validDf.drop(validDf.columns[[0]], 1)
print(validDf)

                                                 tweet
0    #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1    #ConstitutionDay is revered by Conservatives, ...
2    #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
3    #Watching #Boomer getting the news that she is...
4    #NoPasaran: Unity demo to oppose the far-right...
..                                                 ...
855  #DespicableDems lie again about rifles. Dem Di...
856  #MeetTheSpeakers 🙌 @USER will present in our e...
857  3 people just unfollowed me for talking about ...
858  #WednesdayWisdom Antifa calls the right fascis...
859      #Kavanaugh typical #liberals , #Democrats URL

[860 rows x 1 columns]


In [47]:
data_top = df.head()
print(data_top)

                                               tweet subtask_a
0  @USER She should ask a few native Americans wh...       OFF
1  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF
2  Amazon is investigating Chinese employees who ...       NOT
3  @USER Someone should'veTaken" this piece of sh...       OFF
4  @USER @USER Obama wanted liberals &amp; illega...       NOT


In [49]:
text = np.array(df['tweet'])
offen = np.array(df['subtask_a'])

from sklearn.model_selection import train_test_split
text_train, text_test, offen_train, offen_test = train_test_split(text, offen, test_size=.2, random_state=4)

validTweets = np.array(validDf['tweet'])

In [6]:
from sklearn import metrics
import numpy as np
import pandas as pd

def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
                        
def train_predict_display_model(classifier, 
                        train_features=text_train, 
                        test_features=text_test, 
                        train_labels=offen_train, 
                        test_labels=offen_test):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    get_metrics(test_labels,predictions)   


In [34]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.preprocessing import StandardScaler
pipe = Pipeline([
    
    ('feats', FeatureUnion([
    ('tfidf', TfidfVectorizer()),
    ('cvec', CountVectorizer()),
    ])),
   
    
    ('lr' ,LogisticRegression(solver='lbfgs', penalty='l2'))
])

In [37]:
pipe.fit(text,offen)

/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('feats',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('tfidf',
                                                 TfidfVectorizer(analyzer='word',
                                                                 binary=False,
                                                                 decode_error='strict',
                                                                 dtype=<class 'numpy.float64'>,
                                                                 encoding='utf-8',
                                                                 input='content',
                                                                 lowercase=True,
                                                                 max_df=1.0,
                                                                 max_features=None,
                                                                 min_df=1,
                        

In [17]:
# List of parameters
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaler', 'feats', 'lr', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'feats__n_jobs', 'feats__transformer_list', 'feats__transformer_weights', 'feats__verbose', 'feats__tfidf', 'feats__cvec', 'feats__tfidf__analyzer', 'feats__tfidf__binary', 'feats__tfidf__decode_error', 'feats__tfidf__dtype', 'feats__tfidf__encoding', 'feats__tfidf__input', 'feats__tfidf__lowercase', 'feats__tfidf__max_df', 'feats__tfidf__max_features', 'feats__tfidf__min_df', 'feats__tfidf__ngram_range', 'feats__tfidf__norm', 'feats__tfidf__preprocessor', 'feats__tfidf__smooth_idf', 'feats__tfidf__stop_words', 'feats__tfidf__strip_accents', 'feats__tfidf__sublinear_tf', 'feats__tfidf__token_pattern', 'feats__tfidf__tokenizer', 'feats__tfidf__use_idf', 'feats__tfidf__vocabulary', 'feats__cvec__analyzer', 'feats__cvec__binary', 'feats__cvec__decode_error', 'feats__cvec__dtype', 'feats__cvec__encoding', 'feats__cvec__input', 'feats__cvec__lowercase', 'feats__cvec__ma

In [29]:
pipe_parms = [{
    'feats__tfidf__max_features' : [500,4000],
    'feats__tfidf__ngram_range' : [(1,1),(1,3)],
    'feats__cvec__max_features' : [50,500],
    'feats__cvec__ngram_range' : [(1,1),(1,3)],
}]

In [10]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(pipe, param_grid= pipe_parms, cv=5)

In [35]:
train_predict_display_model(classifier=pipe)

/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy: 0.7526
Precision: 0.744
Recall: 0.7526
F1 Score: 0.7437


In [50]:
pipe.predict(validTweets)

array(['OFF', 'NOT', 'NOT', 'NOT', 'NOT', 'OFF', 'OFF', 'NOT', 'NOT',
       'NOT', 'NOT', 'OFF', 'NOT', 'NOT', 'OFF', 'OFF', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'OFF', 'NOT',
       'NOT', 'NOT', 'OFF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'NOT', 'NOT',
       'NOT', 'NOT', 'OFF', 'OFF', 'NOT', 'NOT', 'OFF', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'OFF', 'OFF', 'OFF', 'NOT', 'NOT',
       'OFF', 'NOT', 'NOT', 'NOT', 'OFF', 'NOT', 'NOT', 'NOT', 'NOT',
       'OFF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'OFF', 'NOT', 'NOT', 'NOT', 'OFF', 'NOT', 'OFF', 'OFF', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'OFF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'OFF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT',

In [30]:
gs.fit(text_train,offen_train)

/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iter

/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iter

/home/david/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('feats',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('tfidf',
                                                                        TfidfVectorizer(analyzer='word',
                                                                                        binary=False,
                                                                                        decode_error='strict',
                                                                                        dtype=<class 'numpy.float64'>,
                                                                                        encoding='utf-8',
                                                                                        input='content',
                                                           

In [31]:
print(gs.best_params_)    
print(gs.best_score_)

{'feats__cvec__max_features': 500, 'feats__cvec__ngram_range': (1, 1), 'feats__tfidf__max_features': 2000, 'feats__tfidf__ngram_range': (1, 1)}
0.7553814199395771
